In [ ]:
# !pip install absl-py nltk rouge

In [ ]:
# from prompting.rewards import calculate_rouge_scores


In [1]:
from rouge import Rouge
import pandas as pd

rouge = Rouge()
# Define your reference and hypothesis texts
reference_text = ["The capital of Texas is Austin.", "Austin is the capital of Texas."]
hypothesis_text = ["Austin is the capital of Texas.", "The capital of Texas is Austin."]

# Calculate the Rouge score
rouge_score = rouge.get_scores(hypothesis_text, reference_text)

rouge_score

[{'rouge-1': {'r': 0.8333333333333334,
   'p': 0.8333333333333334,
   'f': 0.8333333283333335},
  'rouge-2': {'r': 0.4, 'p': 0.4, 'f': 0.3999999950000001},
  'rouge-l': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}},
 {'rouge-1': {'r': 0.8333333333333334,
   'p': 0.8333333333333334,
   'f': 0.8333333283333335},
  'rouge-2': {'r': 0.4, 'p': 0.4, 'f': 0.3999999950000001},
  'rouge-l': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}}]

In [ ]:
# dict of dict to list of dict
pd.json_normalize(rouge_score)

In [ ]:
' '.join(reversed("The capital of Texas is Austin.".split()))

In [ ]:
hypotheses = [
    "Austin",
    "The capital of Texas is Austin.",
    "The capital of Texas is not Austin.",
    "The capital of Texas is a city.",
    ' '.join(reversed("The capital of Texas is Austin.".split())),
    "Texas has no capital.",
    "The capital of Texas is Houston.",
    "This is a great question. The capital of Texas is Austin.",
    "The answer is Austin.",
    "The capital of Texas is Austin, Texas.",
    "In Texas, the capital is Austin.",
    "Surely you know that the capital of Texas is Austin.",
    "The capital of Texas is Austin, Texas, USA.",
    "Based on my research, the capital of Texas is Austin.",
    "The capital city of Texas is Austin.",  
    "The capital city of Texas is Austin."+'.'*15,
    "The capital city of Texas is Austin. "*5, 
    "The capital city of Texas is Austin. "*10
    
]

references = [
    "The capital of Texas is Austin.",
    "Austin is the capital of Texas.",
    "The capital is Austin."
]

results = []
for hypothesis in hypotheses:
    for reference in references:
        rouge_score = rouge.get_scores(hypothesis, reference)
        frame = pd.json_normalize(rouge_score)
        frame["hypothesis"] = hypothesis
        frame['hypothesis_length'] = len(hypothesis.split())
        frame["reference"] = reference
        frame['reference_length'] = len(reference.split())
        results.append(frame)
        
        
df = pd.concat(results)    
df

In [ ]:
df.filter(regex='hypothesis|reference').columns

In [ ]:
df_long = df.melt(id_vars=df.filter(regex='hypothesis|reference').columns, var_name="metric", value_name="score")
df_long['type'] = df_long['metric'].str.split(".").str[-1]

# difference in words
df_long['length_difference'] = df_long['hypothesis_length'] - df_long['reference_length']

# relative difference in words (kind of like %)
df_long['length_difference_rel'] = df_long['length_difference'].abs() / df_long['reference_length']

# df_long['score_scaled'] = df_long['score'] * (1-df_long['length_difference_rel'].abs())


import numpy as np
df_long['score_scaled'] = df_long['score'] * np.exp(-df_long['length_difference_rel']**2)

df_long

In [ ]:
x = np.linspace(0, 3, 1000)
y = np.exp(-x**2)
px.line(x=x, y=y, width=800, height=600, template="plotly_white")

In [ ]:
import plotly.express as px

px.histogram(df_long, x="score", facet_col="metric",color="type",
             facet_col_wrap=3,
            width=800, height=600, template='plotly_white'
)

In [ ]:
px.scatter(df_long, x="hypothesis_length", y='score_scaled',
           facet_col="metric", color="type", opacity=0.4,
           hover_data=['hypothesis', 'reference'],
             facet_col_wrap=3,
            width=800, height=600, template='plotly_white'
)

In [ ]:
px.scatter(df_long, x="reference_length", y='score_scaled',
           facet_col="metric", color="type", opacity=0.4,
           hover_data=['hypothesis', 'reference'],
           
             facet_col_wrap=3,
            width=800, height=600, template='plotly_white'
)

In [ ]:
px.scatter(df_long, x="length_difference_rel", y='score_scaled',
           facet_col="metric", color="type", opacity=0.4,
           hover_data=['hypothesis', 'reference','score','score_scaled'],
           
             facet_col_wrap=3,
            width=800, height=600, template='plotly_white'
)

In [ ]:
px.scatter(df_long, x="length_difference", y='score',
           facet_col="metric", color="type", opacity=0.4,
           hover_data=['hypothesis', 'reference'],
           
             facet_col_wrap=3,
            width=800, height=600, template='plotly_white'
)

In [ ]:
px.scatter(df_long, x="hypothesis_length", y='reference_length',
           facet_col="metric", color="score", opacity=0.4,
             facet_col_wrap=3, color_continuous_scale='BlueRed',
            width=800, height=600, template='plotly_white'
)

In [ ]:
!pip install -U angle-emb

In [2]:
from angle_emb import AnglE

angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
vec = angle.encode('hello world', to_numpy=False)
print(vec)
vecs = angle.encode(['hello world1', 'hello world2'], to_numpy=False)
print(vecs)


tensor([[ 0.5039,  0.5862,  0.3513,  ..., -0.1364, -0.1045,  0.3366]],
       device='cuda:0')
tensor([[ 0.7404,  0.5638,  0.1595,  ...,  0.1838, -0.2180,  0.0240],
        [ 0.4121,  0.9849,  0.7677,  ..., -0.4061, -0.3609,  0.1150]],
       device='cuda:0')


In [7]:
from torch.nn.functional import cosine_similarity
# Assuming you have two sentence embeddings stored in `embedding1` and `embedding2`
distance = cosine_similarity(vec, vecs)
print(distance)


tensor([0.8936, 0.8851], device='cuda:0')


In [ ]:
hypothesis_embeddings = angle.encode(hypotheses, to_numpy=False)

hypothesis_embeddings


In [ ]:

reference_embeddings = angle.encode(references, to_numpy=False)

reference_embeddings

In [ ]:
r = []

for i, emb1 in enumerate(hypothesis_embeddings):
    similarities = cosine_similarity(emb1.reshape(1,-1), reference_embeddings)
    for j, score in enumerate(similarities):
        r.append({'hyp': hypotheses[i], 'ref': references[j], 'score': score.item()})
        
    r.append({'hyp': hypotheses[i], 'ref': 'mean', 'score': similarities.mean().item()})
    r.append({'hyp': hypotheses[i], 'ref': 'max', 'score': similarities.max().item()})
    

pd.DataFrame(r).head(30)

In [ ]:
hypothesis_embeddings.shape, reference_embeddings.shape

In [ ]:
hypothesis_embeddings[0].shape

In [ ]:
"""
rewards = ROUGE + Relevance + LengthMatch + Time

ROUGE: Measure of how similar the generated text is to the reference text
PROS: 
- Easy to implement
- fast to compute
CONS: 
- Does not take into account the meaning of the text
- Does not take into account the grammar of the text (i.e word order, repetition, etc)

Relevance: Measure of how relevant the generated text is to the input text using sentence embeddings
PROS:
- Takes into account the meaning of the text
- Takes into account the grammar of the text(?)
CONS:
- Slow to compute
- Susceptible to noise in the input text

"""

In [ ]:
phrases = [
    "In Inazuma Eleven GO 2: Chrono Stone, can you explain how the time travel mechanism works and how it allows players to manipulate the game's narrative and gameplay elements?",
    "In Inazuma Eleven GO 2: Chrono Stone, the concept of time travel is introduced as a major plot element. The story follows the protagonist, Arion Sherwind, as he travels back in time to prevent a catastrophic event that threatens the future of soccer. Arion is aided by his childhood friend, Tenma Matsukaze, who also happens to be the main character from the previous game, Inazuma Eleven GO.",
]

phrases_embeddings = angle.encode(phrases, to_numpy=False)

cosine_similarity(phrases_embeddings[0].reshape(1,-1), phrases_embeddings[1].reshape(1,-1))

In [4]:
categories = [
    'Physics',
    'People',
    'Lifestyle',
    'Travel',
    'Cooking',
    'Geography',
    'Music',
    'Movies',
    'Sports',
    'Politics',
    'History',
    'Business',
    'Mathematics',
    'Science',
    'Technology',
    'Art',
    'Literature',
    'Philosophy',
    'Religion and Spirituality',
    'Health and Fitness',
]

import time

t0 = time.time()
categories_embeddings = angle.encode(categories, to_numpy=False)
print(f'Holy shit that took only {time.time()-t0:.3f} seconds?!')

Holy shit that took only 0.019 seconds?!


In [5]:
documents = [
    'John is a very good football player. He plays for the New York Giants.',
    'Plastic is a material consisting of any of a wide range of synthetic or semi-synthetic organic compounds that are malleable and can be molded into solid objects.',
    'The United States of America (USA), commonly known as the United States (U.S. or US), or America, is a country primarily located in North America, consisting of 50 states, a federal district, five major self-governing territories, and various possessions.',
    'The 2019–20 coronavirus pandemic is an ongoing pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The outbreak was first identified in Wuhan, Hubei, China, in December 2019, and was recognized as a pandemic by the World Health Organization (WHO) on 11 March 2020.',
    'Salvador Domingo Felipe Jacinto Dalí i Domènech, 1st Marquess of Dalí de Púbol (11 May 1904 – 23 January 1989), known professionally as Salvador Dalí (/ˈdɑːli, dɑːˈliː/; Catalan: [səlβəˈðo ðəˈli], Spanish: [salβaˈðoɾ ðaˈli]), was a Spanish Surrealist artist renowned for his technical skill, precise draftsmanship and the striking and bizarre images in his work. Born in Figueres, Catalonia, Dalí received his formal education in fine arts at Madrid.',
    'Children are not little adults. Pound for pound, kids are exposed to more contaminants in air, water, food, and personal care products than adults. Immature organ systems are often less capable of fending off chemical assaults. Subtle damage to developing bodies may lead to disease later in life.',
    'Dirac fermions are fermions which occur as quasiparticles in condensed matter. They were first discussed by Paul Dirac in the context of electrons in graphene, which are described by the massless Dirac equation. Dirac fermions have since been realized experimentally in graphene and topological insulators, and are thought to occur in other materials.',
    'If you believe in God, you believe in the existence of a supreme being, or God, who is worthy of our worship. If you believe in God, you believe that there is a supreme being who is worthy of our worship.',
    'Singapore, officially the Republic of Singapore, is a sovereign island city-state in maritime Southeast Asia. It lies about one degree of latitude (137 kilometres or 85 miles) north of the equator, off the southern tip of the Malay Peninsula, bordering the Straits of Malacca to the west, the Riau Islands (Indonesia) to the south, and the South China Sea to the east.',
    'Robin Williams, in full Robin McLaurin Williams, (born July 21, 1951, Chicago, Illinois, U.S.—died August 11, 2014, Tiburon, California), American comedian and actor known for his manic stand-up routines and his diverse film performances. He won an Academy Award for his role in Good Will Hunting (1997).',
    'The battle of Hastings was fought on 14 October 1066 between the Norman-French army of William, the Duke of Normandy, and an English army under the Anglo-Saxon King Harold Godwinson, beginning the Norman conquest of England. It took place approximately 7 miles (11 kilometres) northwest of Hastings, close to the present-day town of Battle, East Sussex, and was a decisive Norman victory.',
]

documents_embeddings = angle.encode(documents, to_numpy=False)

In [10]:
def get_tags(document, categories_embeddings, n=1, threshold=0.5):
    document_embedding = angle.encode(document, to_numpy=False)
    similarities = cosine_similarity(document_embedding.reshape(1,-1), categories_embeddings)
    if n is not None:
        tags = [categories[i] for i in similarities.argsort()[-n:]]
    else:
        tags = [categories[i] for i, similarity in enumerate(similarities) if similarity >= threshold]
    if not tags:
        closest_match = similarities.argmax()
        print(f'No tags found for document. Closest match: {categories[closest_match]}')
        tags = ['Other']
    return tags

for doc in documents:
    print(doc)
    t0 = time.time()
    print(get_tags(doc, categories_embeddings,n=None))
    print(f'Holy shit that took only {time.time()-t0:.3f} seconds?!')
    print()


John is a very good football player. He plays for the New York Giants.
['Sports']
Holy shit that took only 0.019 seconds?!

Plastic is a material consisting of any of a wide range of synthetic or semi-synthetic organic compounds that are malleable and can be molded into solid objects.
['Science', 'Technology']
Holy shit that took only 0.017 seconds?!

The United States of America (USA), commonly known as the United States (U.S. or US), or America, is a country primarily located in North America, consisting of 50 states, a federal district, five major self-governing territories, and various possessions.
['People', 'Geography']
Holy shit that took only 0.017 seconds?!

The 2019–20 coronavirus pandemic is an ongoing pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The outbreak was first identified in Wuhan, Hubei, China, in December 2019, and was recognized as a pandemic by the World Health Organization (WHO) on 11 Mar